In [ ]:
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

In [ ]:
class PipesDataset(Dataset):
    def __init__(self, path: str):
        self.path = path
        curr_dir = os.getcwd()
        data_path = os.path.join(curr_dir, path)
        self.df = pd.read_csv(data_path, dtype=str)  # dataframe

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        all = self.df.iloc[idx]

        state = all.iloc[0]
        action = all.iloc[1]

        # Create a list, where each entry in the list is an int
        # the list as a whole represents the state of the board
        state_int_list = [int(x) for x in state]
        state_tensor = torch.tensor(state_int_list)

        action_int_list = [int(x) for x in action]
        action_tensor = torch.tensor(action_int_list)

        return (state_tensor, action_tensor)


train_pipes = DataLoader(PipesDataset("data/train.csv"), batch_size=64, shuffle=True)
test_pipes = DataLoader(PipesDataset("data/test.csv"), batch_size=64, shuffle=True)
train_features, train_labels = next(iter(train_pipes))
test_features, test_labels = next(iter(test_pipes))

In [ ]:
class PipesPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
device = (
    torch.accelerator.current_accelerator().type
    if torch.accelerator.is_available()
    else "cpu"
)
print(f"Using {device} device")

n = 4
model = PipesPredictor(input_size=4*(n**2), hidden_size=128, output_size=n**2).to(device)

learning_rate = 1e-3
batch_size = 64
epochs = 15

# Initialize the loss function
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device).float(), y.to(device).float()
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    total_correct = 0  # total correct label predictions
    total_labels = 0   # total number of labels across all samples

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device).float()

            logits = model(X)
            loss = loss_fn(logits, y)
            test_loss += loss.item()

            # Convert logits to probabilities and then to a 0/1 mask.
            probs = torch.sigmoid(logits)
            predicted_mask = (probs >= 0.5).float()

            # Count all correct label predictions (element-wise comparison)
            total_correct += (predicted_mask == y).float().sum().item()
            total_labels += y.numel()  # count of all individual labels

    avg_loss = test_loss / num_batches
    # Calculate average per-label accuracy as a percentage.
    label_accuracy = (total_correct / total_labels) * 100
    print(f"Test Avg loss: {avg_loss:.4f}, Average Label Accuracy: {label_accuracy:.2f}%")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_pipes, model, loss_fn, optimizer)
    test_loop(test_pipes, model, loss_fn)
print("Done!")
model_file_name = "model.pth"
curr_dir = os.getcwd()
model_file_path = os.path.join(curr_dir, model_file_name)
torch.save(model.state_dict(), model_file_path)

In [ ]:
model_file_name = "model.pth"
curr_dir = os.getcwd()
model_file_path = os.path.join(curr_dir, model_file_name)
model.load_state_dict(torch.load(model_file_path, weights_only=True))
model.eval()

In [ ]:
global visited
visited = {}  # Changed from set to dictionary mapping states to sets of actions


initial = "0110101010110100010101101110000110101010001110111000001101000100"
goal = "0110010101110001101001101011001010101010110010111000110000011000"


def pick_move(state):
    global visited
    
    # convert the state to integers
    state_int_list = [int(x) for x in state]
    # convert the state to a tensor
    state_tensor = torch.tensor(state_int_list).to(device).float()
    # get the predicted move from the neural network
    prob = model(state_tensor)
    results = torch.topk(prob, 16).indices
    
    # Initialize the visited state if we haven't seen it before
    if state not in visited:
        visited[state] = set()
        
    # Try each of the top moves
    new_state = state
    result = 0
    for r in results:
        result = int(r)
        # Skip if we've already tried this move for this state
        if result in visited[state]:
            continue
            
        new_state = pipe_rotate_binary(result, state)
        # Mark this move as tried for this state
        visited[state].add(result)
        break
        
    if new_state == state:
        raise Exception("No valid moves available")
    return new_state, result


def pipe_rotate_binary(pipe: int, board: str):
    """
    Takes a binary representation of a board of pipes as a string, and a pipe to rotate. Outputs a binary representation of the board after rotating the pipe.

    :params pipe: The pipe to rotate
    :params board: Binary representation of the board as a string

    """
    # each pipe has 4 values associated to it, so pipe n starts at index 4 * n
    start_index = 4 * pipe
    up = board[start_index]
    right = board[start_index + 1]
    down = board[start_index + 2]
    left = board[start_index + 3]

    # rotate clockwise
    new_board = (
        board[:start_index] + left + up + right + down + board[start_index + 4 :]
    )

    return new_board


state = initial
visited[initial] = set()  # Initialize the visited dictionary
moves = 0
while state != goal:
    state, result = pick_move(state)
    print(state, result)
    moves += 1
print(f"moves: {moves}")